### Startup imports

In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import slurm_manager
from autoqchem.sge_manager import sge_manager
from autoqchem.helper_classes import slurm_status,sge_status
from autoqchem.draw_utils import draw

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the job manager

The job manager is going to manage what jobs you have currently running or waiting to run. It caches your information, so you can kill the notebook, turn-off your computer, or go on vacation, and still pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Use the ```slurm``` manager for running jobs on ```della``` at Princeton and the ```sge``` manager for running jobs on ```hoffman2``` at UCLA.

Note: it won't ask you to login unless you execute a method that requires remote access.

In [2]:
#sm=slurm_manager(user='zuranski', host='della.princeton.edu')
sm=sge_manager(user='wgee', host='hoffman2.idre.ucla.edu')

### Provide some SMILES string(s)

The ```my_smiles``` variable is a list of strings (a Python list can be of any size)

In [31]:
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(Br)cc1",
]

For a list of SMILES, you can also read them from a csv file, assuming that the filename is ```smiles_strings.csv```, the strings are in the first column, and there are no headers

In [7]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will make 3D conformations for every SMILES string using a ```for``` loop

In [32]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, num_conf=10))

Draw some molecules

In [33]:
draw(mols[0].mol)
draw(mols[1].mol)

interactive(children=(Dropdown(description='confId', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Output(…

interactive(children=(Dropdown(description='confId', options=(0,), value=0), Output()), _dom_classes=('widget-…

<function autoqchem.draw_utils._graph_conf(m, confId=0, energies=[])>

### Create Gaussian jobs for each conformation

Select settings such as the functional, basis set, and solvent

In [35]:
for mol in mols[0:2]:
    sm.create_jobs_for_molecule(mol, theory="B3LYP", light_basis_set='6-31G**', solvent='methanol')

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 10 conformations.


### View the status of all jobs in the workflow


In [37]:
sm.get_job_stats(split_by_can=True)

status,created
can,
C(C1C(C(C(C(O1)O)O)O)O)O,10


### Submit the jobs to the server 

If you are not connected, it will ask for password + Duo (`della`) or password (`hoffman2`)

In [38]:
sm.submit_jobs()

INFO:autoqchem.sge_manager:Submitting 10 jobs.
INFO:autoqchem.sge_manager:Creating connection to hoffman2.idre.ucla.edu as wgee
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (password) successful!
INFO:autoqchem.sge_manager:Connected to hoffman2.idre.ucla.edu as wgee.
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:autoqchem.sge_manager:Submitted job f3acd84ad1cddc42b1c76dcdb37d544d, job_id: 7357811.
INFO:autoqchem.sge_manager:Submitted job 6a4a352e2a9423622ee71ddd8cf0979b, job_id: 7357815.
INFO:autoqchem.sge_manager:Submitted job 79f17c43b84edcdadd9132a09fffea3b, job_id: 7357819.
INFO:autoqchem.sge_manager:Submitted job c936e56c7d5c54d289a4db58f8344055, job_id: 7357824.
INFO:autoqchem.sge_manager:Submitted job ba77b2860aaf978eeebdc6ada1e94811, job_id: 7357828.
INFO:autoqchem.sge_manager:Submitted job 7176e2d87aacfcc39234439d331d614e, job_id: 7357833.
INFO:autoqchem.sge_manager:Submitted j

Check the status of the jobs on the server

In [39]:
#sm.squeue(summary=False)   # with slurm on `della`
sm.qstat(summary=False)     # with sge on `hoffman2`

job-ID     prior   name       user         state submit/start at     queue                          jclass                         slots ja-task-ID 
------------------------------------------------------------------------------------------------------------------------------------------------
   7357811 0.50500 WQZGKKKJIJ wgee         r     05/19/2023 11:23:23 pod_ib100_apollo2.q@n6676                                         4        
   7357815 0.50500 WQZGKKKJIJ wgee         r     05/19/2023 11:23:23 pod_ib100_apollo2.q@n6676                                         4        
   7357819 0.50500 WQZGKKKJIJ wgee         r     05/19/2023 11:23:30 pod_apollo_6140.q@n6643                                           4        
   7357824 0.50500 WQZGKKKJIJ wgee         r     05/19/2023 11:23:30 pod_apollo_6140.q@n7672                                           4        
   7357828 0.50500 WQZGKKKJIJ wgee         r     05/19/2023 11:23:30 pod_apollo_6140.q@n7670                                  

Alternatively, view summary of job status on the server
(may take a bit longer for many jobs as asks server about each separately)

In [40]:
#sm.squeue(summary=True)    # with slurm on `della`
sm.qstat(summary=True)      # with sge on `hoffman2`

,Count
running,10
queued,0


### Check workflow status at any time


In [3]:
sm.get_job_stats(split_by_can=True)

status,submitted
can,
C(C1C(C(C(C(O1)O)O)O)O)O,10


### Retrieve jobs

If there are some finished jobs, the log files will be checked for completion and downloaded locally.

In [4]:
sm.retrieve_jobs()

INFO:autoqchem.sge_manager:Creating connection to hoffman2.idre.ucla.edu as wgee
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (password) successful!
INFO:autoqchem.sge_manager:Connected to hoffman2.idre.ucla.edu as wgee.
INFO:autoqchem.sge_manager:There are 0 running/pending jobs, 10 finished jobs.
INFO:autoqchem.sge_manager:Retrieving log files of finished jobs.
INFO:paramiko.transport.sftp:[chan 2] Opened sftp connection (server version 3)
INFO:autoqchem.sge_manager:10 jobs finished successfully (all Gaussian steps finished normally). 0 jobs failed.


### Resubmit incomplete jobs

In [5]:
sm.resubmit_incomplete_jobs()

INFO:autoqchem.sge_manager:There are no incomplete jobs to resubmit.


### Upload molecules to the database

In [6]:
sm.upload_done_molecules_to_db(tags=["test_WCG"])

INFO:autoqchem.sge_manager:There are 1 finished molecules ['C(C1C(C(C(C(O1)O)O)O)O)O'].
INFO:autoqchem.sge_manager:Molecule C(C1C(C(C(C(O1)O)O)O)O)O has 0 / 10 duplicate conformers.
INFO:autoqchem.sge_manager:Removing 0 / 10 jobs and log files that contain duplicate conformers.
INFO:autoqchem.sge_manager:Uploaded descriptors to DB for smiles: C(C1C(C(C(C(O1)O)O)O)O)O, number of conformers: 10, DB molecule id 6467cfbfded247f07276fe3c.


You can now find the computed descriptors for your molecules at https://autoqchem.org/.

## Other helpful commands

### Remove molecules from job manager

Useful if need to redo a molecule with same Gaussian configuration before uploading to database

In [26]:
sm.remove_jobs(sm.get_jobs(can="C(C1C(C(C(C(O1)O)O)O)O)O"))

### Cancel all running/queued jobs

Beware that all of a user's jobs will be canceled (including jobs submitted independently of Auto-QChem)

In [23]:
#sm.scancel()   # with slurm on `della`
sm._qdel()      # with sge on `hoffman2`

wgee has registered the job 7357214 for deletion
wgee has registered the job 7357215 for deletion
wgee has registered the job 7357217 for deletion
wgee has registered the job 7357218 for deletion
wgee has registered the job 7357219 for deletion
wgee has registered the job 7357220 for deletion
wgee has registered the job 7357221 for deletion
wgee has registered the job 7357222 for deletion
wgee has registered the job 7357290 for deletion
wgee has registered the job 7357291 for deletion
wgee has registered the job 7357292 for deletion
wgee has registered the job 7357293 for deletion
wgee has registered the job 7357294 for deletion
wgee has registered the job 7357295 for deletion
wgee has registered the job 7357296 for deletion
wgee has registered the job 7357297 for deletion
wgee has registered the job 7357298 for deletion
wgee has registered the job 7357299 for deletion
